In [150]:
import os
from pathlib import Path 
import pandas as pd
import pickle

home = ".."

In [151]:
with open("../../sentimentModelDump", "rb") as fp:
    model, vec = pickle.load(fp)

print(model, vec)

MultinomialNB() CountVectorizer()


## Spark Context 

In [152]:
from pyspark import SparkConf
from pyspark.context import SparkContext

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

## Streaming Context 

In [153]:
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession

ssc = StreamingContext(sc, 1)
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [154]:
PORT=4015 # Change this to a unique port before running individually
HOST="localhost"

print("Run this command at the terminal and type in words and hit enter periodically:")
print(f"nc -lk {PORT}")

Run this command at the terminal and type in words and hit enter periodically:
nc -lk 4015


In [155]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import traceback

# Lazily instantiated global instance of SparkSession
def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]

ssc = StreamingContext(sc, 1)
ssc.checkpoint("checkpoint")

In [156]:
testDF = pd.DataFrame({"Text" : ["This is very negative tweet"]})

In [157]:
model.predict(vec.transform(testDF))

array([0], dtype=int32)

In [158]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType


schema = StructType([ \
    StructField("ItemID", IntegerType(), True), \
    StructField("Sentiment", IntegerType(), True), \
    StructField("SentimentText",StringType(),True)
  ])

## Lets just check to make sure that the model still works...

1: Positive Sentiment <br>
0: Negative Sentiment

In [159]:
test_spark_df = spark.read.schema(schema).csv(f'{home}/csc-369-student/data/twitter_sentiment_analysis/historical/xb*')
historical_test_data = test_spark_df.toPandas()
test_tf_sparse = vec.transform(historical_test_data['SentimentText'])
print("Accuracy on new historical test data:",sum(model.predict(test_tf_sparse) == historical_test_data['Sentiment'])/len(historical_test_data))

Accuracy on new historical test data: 0.7223846153846154


# Scratch
def test(line):
    df = pd.DataFrame({"Tweet" : [line]})
    pred = model.predict(vec.transform(testDF))


    return spark.createDataFrame([("Positive:" if pred == 1 else "Negative", line)])
    # line = "{} {}".format(
    #             "Positive:" 
    #             if pred == 1 else 
    #             "Negative", 
                
    #             line)

    

    return line

# tweets = lines.map(test)
tweets = lines.map(lambda line: pd.DataFrame({"Tweet" : [line]}))
# tweets = lines.map(lambda line: model.predict(vec.transform(testDF)))
tweets = lines.map(lambda line: model.predict(vec.transform(line)))
# tweets = lines.map(lambda line: line + "Positive:" if model.predict(vec.transform(line)) == 1 else "Negative")




In [160]:
def line_to_row2(line):
        print("HELLO WORLD")
        print("Line: ", line)
        print("SentimentText: " , line)
        return Row(SentimentText=line)

def process(time, rdd):


    print("========= %s =========" % str(time))
    if rdd.isEmpty():
        return
    else:
        spark = getSparkSessionInstance(rdd.context.getConf())
        print('hello')
        print("Starting")
        rowRdd = rdd.map(line_to_row2)
            
        # print("row RDD: ", rowRdd)
        df_rows = spark.createDataFrame(rowRdd)
        print("df_rows: ", df_rows)
        df_pandas = df_rows.toPandas()
        print("I AM ALIVE")
        print("df_pandas", df_pandas.values())
        # model.pre
    # except Exception:
    #     print(traceback.format_exc())

In [161]:
HOST = "localhost"
PORT = 4015
lines = ssc.socketTextStream(HOST, PORT)

lines.foreachRDD(process)
# tweets.pprint()

ssc.start()
import time; time.sleep(30)
#ssc.awaitTerminationOrTimeout(60) # wait 60 seconds
ssc.stop(stopSparkContext=False)

========= 2021-11-17 18:13:59 =========
========= 2021-11-17 18:14:00 =========
========= 2021-11-17 18:14:01 =========
========= 2021-11-17 18:14:02 =========
========= 2021-11-17 18:14:03 =========
========= 2021-11-17 18:14:04 =========
========= 2021-11-17 18:14:05 =========
========= 2021-11-17 18:14:06 =========
========= 2021-11-17 18:14:07 =========
========= 2021-11-17 18:14:08 =========
========= 2021-11-17 18:14:09 =========
========= 2021-11-17 18:14:10 =========
========= 2021-11-17 18:14:11 =========
========= 2021-11-17 18:14:12 =========
hello
Starting
df_rows:  DataFrame[SentimentText: string]
========= 2021-11-17 18:14:13 =========
hello
Starting
df_rows:  DataFrame[SentimentText: string]
========= 2021-11-17 18:14:14 =========
hello
Starting
df_rows:  DataFrame[SentimentText: string]
========= 2021-11-17 18:14:15 =========
========= 2021-11-17 18:14:16 =========
========= 2021-11-17 18:14:17 =========
========= 2021-11-17 18:14:18 =========
========= 2021-11-17 18:14

In [113]:
def test(line):
    print("hello world")
    return "hello world"
    # df = pd.DataFrame({"Tweet" : [line]})
    # pred = model.predict(vec.transform(testDF))


    # return spark.createDataFrame([("Positive:" if pred == 1 else "Negative", line)])
    # # line = "{} {}".format(
    # #             "Positive:" 
    # #             if pred == 1 else 
    # #             "Negative", 
                
    # #             line)

HOST = "localhost"
PORT = 4015
lines = ssc.socketTextStream(HOST, PORT)

tweets = lines.map(test)
tweets.pprint()

ssc.start()
import time; time.sleep(30)
#ssc.awaitTerminationOrTimeout(60) # wait 60 seconds
ssc.stop(stopSparkContext=False)

-------------------------------------------
Time: 2021-11-17 18:04:39
-------------------------------------------

-------------------------------------------
Time: 2021-11-17 18:04:40
-------------------------------------------

-------------------------------------------
Time: 2021-11-17 18:04:41
-------------------------------------------

-------------------------------------------
Time: 2021-11-17 18:04:42
-------------------------------------------

-------------------------------------------
Time: 2021-11-17 18:04:43
-------------------------------------------

-------------------------------------------
Time: 2021-11-17 18:04:44
-------------------------------------------

-------------------------------------------
Time: 2021-11-17 18:04:45
-------------------------------------------
hello world

-------------------------------------------
Time: 2021-11-17 18:04:46
-------------------------------------------

-------------------------------------------
Time: 2021-11-17 18:04:4